# December 2016: Advent of Code

## Ben Emery (willcodefortea)

[Advent of Code](https://adventofcode.com/2016) is a series of Christmass based programming puzzels. Here are my solutions.

# Day 0: Getting ready / Common

In [243]:
from collections import Counter, defaultdict, namedtuple
from itertools import islice, combinations, chain
from io import StringIO
from heapq import heappush, heappop
import os
import re
import string
import urllib.request

import numpy as np


def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable."
    it = iter(seq)
    result = tuple(islice(it, n))

    if len(result) == n:
        yield result    

    for elem in it:
        result = result[1:] + (elem,)
        yield result


Point = namedtuple('Point', 'x,y')

OFF = '.'

def Input(day):
    """Fetch the data input from disk or URL."""
    filename = os.path.join('../data/advent2016/input{}.txt'.format(day))
    return open(filename)
    

def manhattan_distance(point1, point2):
    """Absolute distance between two N-dimensional points"""
    assert len(point1) == len(point2)
    dist = 0
    for pair in zip(point1, point2):
        dist += abs(pair[0] - pair[1])
    return dist


def transpose(matrix):
    """Swap a matrix about it's leading diagonal.
    
    Zip returns an iterable where the nth element is the nth element
    from each iterable. So by passing each column as an argument
    to zip we create a matrix where the rows are swapped with
    the columns.
    
    [
        [1, 3, 5],
        [2, 4, 6],
    ]
    
    becomes:
    
    [
        [1, 2],
        [3, 4],
        [5, 6],
    ]
    """
    return zip(*matrix)


cat = ''.join


def neighbours4(x, y):
    return (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)


def a_star(start, h_func, moves, cost=lambda s1, s2: 1):
    """A* implementation.
    
    Finds the shortest sequence of states from to to goal (a state where
    the hueristic function - h_func - is zero). We use a heap
    as our priority queue, and processes those with the smallest
    overall cost first (the cost of the path + the distance to target).
    
    start:  the initial state to explore from
    h_func: hueristic function that gives a "distance" to to target state,
            when this is zero we're done.
    moves:  function that generates all possible states from the supplied state
            (these can go bakckwards, but will never be processed)
    cost:   the cost of moving from ones state to another
    
    return: list of states used to find the final state, wll raise an exception
            if none was found.
    
    """
    # The priority queue that we'll be reading from
    queue  = []
    # We often care about the path taken, so persist the lowest costing path
    # to each state
    previous = {
        start: None
    }    
    # Lookup of state costs, we initialize at zero for the starting state
    costs = {
        start: 0
    }
    
    # Initialize our queue, this is ordered by path cost (f(n) = g(n) + h(n))
    add_to_queue = lambda state: heappush(queue, (costs[state] + h_func(state), state))
    
    # Recursively walk backwards to build the full path
    get_path = lambda state: [] if state is None else get_path(previous[state]) + [state]
    
    # Set the intial position and go!
    add_to_queue(start)
    
    while queue:
        f, state = heappop(queue)
        if h_func(state) ==  0:
            # We're done!
            return get_path(state)
        
        for new_state in moves(state):
            new_cost = costs[state] + cost(state, new_state)
            
            if new_state not in costs or new_cost < costs[new_state]:
                # We've found a new state or a better path
                costs[new_state] = new_cost
                previous[new_state] = state
                # We've modified our costs in some way, we need
                # to explore from this state so add to the heap
                add_to_queue(new_state)

    
    # No solution was found
    raise Exception('No solution for A* was discovered.')

Testing some of our utilities

In [166]:
assert manhattan_distance((0, 0), (3, 4)) == 7, 'Invalid manhattan distance calculation'
print(list(transpose([[1, 3, 5], [2, 4, 6]])))
assert list(transpose([[1, 2], [3, 4]])) == [(1, 3), (2, 4)], 'Invaid matrix transpose'

[(1, 2), (3, 4), (5, 6)]


# [Day 1: No Time for a Taxi Cab](http://adventofcode.com/2016/day/1)

Given a sequence of moves of 'L2' where L means turn left and walk 2 blocks, how many blocks do we end up from the start? Choices:

* Track current location as (x,y) coordinates
* Directions are kept as constants, and rotations are performed by cycling through them
* Moves parsed to (rotation, distance) pairs. Right turns move clockwise, left turns anticlockwise.

In [87]:
L, R = -1, 1
N, E, S, W = range(4)

def rotate(facing, rotation):
    return(facing + rotation) % 4


def shortest_path(input):
    """Shortest distance betwen """
    facing = N
    origin = [0, 0]
    loc = origin[:]
    
    for rotation, distance in parse(input):
        facing = rotate(facing, rotation)
        index = 1 if facing in (N, S) else 0
        sign = 1 if facing in (N, E) else -1
        loc[index] += sign * distance

    return manhattan_distance(origin, loc)
        

def parse(steps):
    """Convert Rotation-Distance to distinct integer pairs."""
    parsed = []
    for step in steps.split(', '):
        rotation = L if step[0] == 'L' else R
        distance = int(step[1:])
        parsed.append((rotation, distance))
    return parsed
    

assert parse("L4, L3, R1") == [(-1, 4), (-1, 3), (1, 1)]
assert rotate(N, L) == W
assert rotate(E, R) == S


shortest_path(Input(1).read())

332

For the second part we need to track which all locations we visit not just those we end up on. Here keep all visited locations in a set, extending with each move.

In [88]:
def first_visited_twice(input):
    """Find the first location visited twice following."""
    facing = N
    origin = (0, 0)
    loc = origin[:]
    
    visited = {loc}

    for rotation, distance in parse(input):
        facing = rotate(facing, rotation)
        index = 1 if facing in (N, S) else 0
        sign = 1 if facing in (N, E) else -1

        for i in range(distance):
            next_location = list(loc[:])
            next_location[index] += sign
            next_location = tuple(next_location)
            
            if next_location in visited:
                return manhattan_distance(origin, next_location)
            loc = next_location
            visited.add(loc)

assert first_visited_twice("R8, R4, R4, R8") == 4
assert first_visited_twice("R8, R4, R4, L8") == None
assert first_visited_twice("R8, R0, R1") == 7
    
first_visited_twice(Input(1).read())

166

# [Day 2: Bathroom Security](https://adventofcode.com/2016/day/2)

We're tasked with following instructions to around a grid, with moves that would leave us out of bounds being ignored. The easiet approach to this is to simply perform each move, and discard it if we end up out of bounds.

In [32]:
Keypad = str.split

keypad = Keypad("""
.....
.123.
.456.
.789.
.....
""")


def follow_instructions(keypad, start, instructions):
    x, y = start
    for instruction in instructions:
        if instruction == 'U' and keypad[x - 1][y] != OFF: x = x - 1
        if instruction == 'D' and keypad[x + 1][y] != OFF: x = x + 1
        if instruction == 'L' and keypad[x][y - 1] != OFF: y = y - 1
        if instruction == 'R' and keypad[x][y + 1] != OFF: y = y + 1
    return (x, y)


def build_code(keypad, input, start):
    pos = start
    code = []
    for instructions in input.split():
        pos = follow_instructions(keypad, pos, instructions)
        x, y = pos
        code.append(keypad[x][y])
    return cat(code)


assert keypad[2][2] == '5'
assert follow_instructions(keypad, (2, 2), 'ULL') == (1, 1)
assert build_code(keypad, 'ULL', (2, 2)) == '1'

build_code(keypad, Input(2).read(), (2, 2))

'38961'

For the second portion of the problem, the keypad changes. We don't need to modify our algorithm at all, just the structure of the kepad.

In [116]:
keypad = Keypad("""
.......
...1...
..234..
.56789.
..ABC..
...D...
.......
""")

build_code(keypad, Input(2).read(), (3, 1))

'46C92'

# [Day 3: Squares With Three Sides](http://adventofcode.com/2016/day/3)

This problem requires validating triangles, a triangle is only valid of the sum of two sides are greater than the third, and so is easy to check.


In [167]:
def is_triangle(sides):
    a, b, c = sorted(sides)
    return c < a + b

# Parse the triangles into a single list
triangles = [
    [int(num) for num in line.split()]
    for line in Input(3)
    if line
]


assert not is_triangle([5, 10, 25])
sum(map(is_triangle, triangles))


1050

Now we must group values by colums, not by rows.
```
A1 A2 A3
B1 B2 B3
C1 C2 C3
```

i.e. `A1` `B1` and `C1` are now a single triangle. To achieve this I rely on the previous data parsing to read the triangles in to rows, and then read from the generator 3 items at a time and construct the appropriate triangles from there.

In [171]:
def translate_triangles():
    """Take 3 triangles at a time and rotate how we group sides."""
    for index in range(0, len(triangles), 3):
        yield from transpose(
            triangles[index:index + 3]
        )
sum(map(is_triangle, translate_triangles()))

1921

# [Day 4: Security Through Obscurity](http://adventofcode.com/2016/day/4)

This problem requires us deterimine if a room is real or a decoy based on the encrypted form of its name. This form is
made of the following components:

    <encrypted_name>-<selector_id>-<[checksum]>
    
Where the encrypted name are lowercase characters and dashes.
    

In [243]:
encryption_re = re.compile('([\w\-]+)-(\d+)\[(\w+)\]')

def parse_encryption(input):
    """Split a room's code into it's distinct chunks."""
    return encryption_re.findall(input)

def is_real_room(room):
    encrypted_name, _, checksum = room
    character_count = Counter(encrypted_name.replace('-', ''))
    preferred_order = sorted(
        character_count.keys(),
        key=lambda c: (-character_count[c], c)
    )
    return cat(preferred_order)[:len(checksum)] == checksum

    
assert parse_encryption('aaaaa-bbb-z-y-x-123[abxyz]') == [('aaaaa-bbb-z-y-x', '123', 'abxyz'), ]
assert is_real_room(('aaaaa-bbb-z-y-x', '123', 'abxyz'))

rooms = parse_encryption(Input(4).read())
real_rooms = [
    room
    for room in rooms
    if is_real_room(room)
]
sum(int(selector_id) for _, selector_id, _ in real_rooms)

137896

In part two we have to decrypt a room's name by rotating characters N times through the alphabet, N being the selector id. Once decrypted we need to find the sector ID that represents where "North pole objects are stored", so filter out results by the expeted keywords.

In [246]:
def decrypt_name(encypted_name, selector_id):
    alphabet = string.ascii_lowercase
    shift_amount = selector_id % len(alphabet)
    from_translation = alphabet + '-'
    to_translation = alphabet[shift_amount:] + alphabet[:shift_amount] + ' '
    trans = str.maketrans(
        from_translation,
        to_translation
    )
    return encypted_name.translate(trans)

assert decrypt_name('qzmt-zixmtkozy-ivhz', 343) == 'very encrypted name'

for room in real_rooms:
    encrypted_name, sector_id, _ = room
    decrypted_name = decrypt_name(encrypted_name, int(sector_id))
    if 'north' in decrypted_name:
        print(decrypted_name, sector_id)

northpole object storage 501


# [Day 5: How About a Nice Game of Chess?](http://adventofcode.com/2016/day/5)

We need to find a password for a door. The password is built from taking the 6th character of the first 8 hashes that have five leading zeros. With each hash being of the form {door_id}{hash_index}.

i.e. for a door_id of 'abc' the first hash found is at index 3231929,

    abc3231929 -> 00000155f8105dff7f56ee10fa9b9abd

so the password character is `1`.

In [38]:
import hashlib


def find_hashes(door):
    i = 0
    
    while True:
        md5 = hashlib.md5(bytes(door + str(i), 'utf-8')).hexdigest()
        if md5.startswith('00000'):
            yield i, md5
        i += 1

def generate_password(door):
    password = ''
    for hash_num, md5 in find_hashes(door):
        password += md5[5]
        print(hash_num, md5, password)
        if len(password) == 8:
            return password

# Uncomment to run test, but it takes some time.
# assert generate_password('abc') == '18f47a30'
door = 'wtnhxymk'
generate_password(door)

3231929 00000155f8105dff7f56ee10fa9b9abd 1
5017308 000008f82c5b3924a1ecbebf60344e00 18
5278568 00000f9a2c309875e05c5a5d09f1b8c4 18f
5357525 000004e597bd77c5cd2133e9d885fe7e 18f4
5708769 0000073848c9ff7a27ca2e942ac10a4c 18f47
6082117 00000a9c311683dbbf122e9611a1c2d4 18f47a
8036669 000003c75169d14fdb31ec1593915cff 18f47a3
8605828 0000000ea49fd3fc1b2f10e02d98ee96 18f47a30
2231254 0000027b9705c7e6fa3d4816c490bbfd 2
2440385 00000468c8625d85571d250737c47b5a 24
2640705 0000013e3293b49e4c78a5b43b21023b 241
3115031 0000040bbe4509b48041007dec6123bd 2414
5045682 00000b11810477f9e49840991fb2151e 2414b
8562236 00000cc461c8945671046cf632be4473 2414bc
9103137 000007c1da6865df78b2c0addf28913d 2414bc7
9433034 00000700ce8beb0a8ffc83fa9986d577 2414bc77


'2414bc77'

The second portion changes the meaning of generated hash, now the 6th character represents the password character and the 7th is the character to use.

In [39]:
def generate_password(door):
    characters = [OFF,] * 8
    for hash_num, md5 in find_hashes(door):
        try:
            position = int(md5[5])
        except ValueError:
            # Invalid position, skip
            continue

        if position > 7:
            # Invalid position, skip
            continue

        character = md5[6]
        if characters[position] is OFF:
            characters[position] = character
            print(hash_num, md5, cat(characters))

        if OFF not in characters:
            return cat(characters)

# Uncomment to run test, but it takes some time.
# assert generate_password('abc') == '05ace8e3'
generate_password(door)

2231254 0000027b9705c7e6fa3d4816c490bbfd ..7.....
2440385 00000468c8625d85571d250737c47b5a ..7.6...
2640705 0000013e3293b49e4c78a5b43b21023b .37.6...
9103137 000007c1da6865df78b2c0addf28913d .37.6..c
13753308 0000050301b17d598b52e2a343b80c95 .37.60.c
13976178 000006fe7545b487de2d003f3d4e1114 .37.60fc
19808390 000003e432ea631581aefcce573d56dd .37e60fc
27712456 00000048d155e2c930602533209b0154 437e60fc


'437e60fc'

# [Day 6: Signals and Noise](http://adventofcode.com/2016/day/6)

Here we have a set of strings and are required to find the most comon value in each column.

The `Counter` object is a dict subclass that will count an iterable. Calling `most_common([n])` returns a tuple of the `n` most common objects and their occurance (in order of highest to lowest).

i.e.

    Counter(a=2, b=3, c=4).most_common(2) == ((a, 2), (b, 3))
    

In [69]:
def most_common(data):
    lines = re.findall('\w+', data)
    cols = transpose(lines)
    return cat(
        Counter(col).most_common(1)[0][0]
        for col in cols
    )

test_data = """
eedadn
drvtee
eandsr
raavrd
atevrs
tsrnev
sdttsa
rasrtv
nssdts
ntnada
svetve
tesnvt
vntsnd
vrdear
dvrsen
enarar
"""

assert most_common(test_data) == 'easter'
most_common(Input(6).read())

'wkbvmikb'

Part two is the same as the first, but instead is the _least_ common value.

In [70]:
def least_common(data):
    lines = re.findall('\w+', data)
    cols = transpose(lines)
    return cat(
        Counter(col).most_common()[-1][0]
        for col in cols
    )

assert least_common(test_data) == 'advent'
least_common(Input(6).read())

'evakwaga'

# [Day 7: Internet Protocol Version 7](https://adventofcode.com/2016/day/7)

Here we're tasked with finding IPv7 addresses that support TLS (transport-layer snooping). IPv7 addresses are of the form `aaa[bbb]ccc[ddd]` with any number of subnets (codes within square brackets) and supernets (outside of square brackets).

In [154]:
subnet_re = re.compile(r'\[(\w+)\]')
supernet_re = re.compile(r'(?:^|\])([a-zA-Z]+)(?:$|\[)')


def has_abba(s):
    for chunk in window(s, 4):
        if chunk[:2] == chunk[:1:-1] and chunk[0] != chunk[1]:
            return True
    return False


def supports_tls(ip):
    subnet_chunks = subnet_re.findall(ip)
    supernet_chunks = supernet_re.findall(ip)

    return not any(map(has_abba, subnet_chunks)) and any(map(has_abba, supernet_chunks))


assert supports_tls('abba[mnop]qrst')
assert not supports_tls('abcd[bddb]xyyx')
assert not supports_tls('aaaa[qwer]tyui')
assert supports_tls('ioxxoj[asdfgh]zxcvbn')

sum(map(supports_tls, Input(7).readlines()))

110

For the second portion, we need to check if a supernet contains a three character palindrome, with its inverse being present in the subnet.

In [153]:
def get_abas(chunks):
    for chunk in chunks:
        for c in window(chunk, 3):
            if c[0] == c[2] and c[0] != c[1]:
                yield cat(c)

                
def supports_ssl(ip):
    subnet_chunks = subnet_re.findall(ip)
    supernet_chunks = supernet_re.findall(ip)
    
    # extract all ABAs
    expected_babs = {
        '{1}{0}{1}'.format(aba[0], aba[1])
        for aba in get_abas(supernet_chunks)
    }
    babs = {c for c in get_abas(subnet_chunks)}
    return len(babs & expected_babs) > 0
    

assert supports_ssl('aba[bab]xyz')
assert not supports_ssl('xyx[xyx]xyx')
assert supports_ssl('aaa[kek]eke')
assert supports_ssl('zazbz[bzb]cdb')

sum(map(supports_ssl, Input(7).readlines()))

242

# [Day 8: Two-Factor Authentication](https://adventofcode.com/2016/day/8)

For this puzzel we're need to rotate arrays and set their values in bulk. Numpy arrays are ideal for this, especially as thir syntax allows us to easily access rows or columns.

In [35]:
def follow_instructions(screen, instructions):
    for instruction in instructions:
        A, B = map(int, re.findall(r'(\d+)', instruction))
        if instruction.startswith('rect'):
            rect(screen, A, B)
        elif instruction.startswith('rotate row'):
            rotate_row(screen, A, B)
        elif instruction.startswith('rotate column'):
            rotate_col(screen, A, B)


def build_screen(width, height):
    return np.zeros([width, height], dtype=np.int)


def to_display(screen):
    rows = [
        cat(['.#'[cell] for cell in row])
        for row in screen
    ]
    return '\n'.join(rows)


def rect(screen, A, B):
    screen[:B, :A] = 1
    

def rotate_col(screen, A, B):
    col = screen[:, A]
    screen[:, A] = np.append(col[-B:], col[:-B])

    
def rotate_row(screen, A, B):
    row = screen[A, :]
    screen[A, :] = np.append(row[-B:], row[:-B])



test_screen = build_screen(3, 7)

test_instructions = [
    'rect 3x2',
    'rotate column x=1 by 1',
    'rotate row y=0 by 4',
    'rotate column x=1 by 1',   
]
follow_instructions(test_screen, test_instructions)
print(to_display(test_screen))

screen = build_screen(6, 50)
instructions = Input(8).readlines()
follow_instructions(screen, instructions)

sum(sum(screen))


.#..#.#
#.#....
.#.....


116

The second portion simply requires us to read the screen.

In [102]:
print(to_display(screen))

#..#.###...##....##.####.#....###...##..####.####.
#..#.#..#.#..#....#.#....#....#..#.#..#.#.......#.
#..#.#..#.#..#....#.###..#....###..#....###....#..
#..#.###..#..#....#.#....#....#..#.#....#.....#...
#..#.#....#..#.#..#.#....#....#..#.#..#.#....#....
.##..#.....##...##..#....####.###...##..####.####.


# [Day 9: Explosives in Cyberspace](http://adventofcode.com/2016/day/9)

In [92]:
matcher = re.compile('[(](\d+)x(\d+)[)]').match


def decompress(compressed):
    re.sub('\s', '', compressed)  # remove any whitespace
    result = []
    i = 0
    
    while i < len(compressed):
        match = matcher(compressed, i)
        if match:
            i = match.end()
            length, multiplier = map(int, match.groups())
            result.append(
                compressed[i:i + length] * multiplier
            )
            i += length
        else:
            result.append(compressed[i])
            i += 1
    return cat(result)
            

assert decompress('ADVENT') == 'ADVENT'
assert decompress('A(1x5)BC') == 'ABBBBBC'
assert decompress('(3x3)XYZ') == 'XYZXYZXYZ'
len(decompress(Input(9).read()))

74532

In part 2 we're required to decompress the compressed chunks, which has a mulitiplicative effect with the parent (hence the explosion). Instead of decompressing the whole file we can count the number of characters.

In [101]:
def decompressed_length(compressed):
    re.sub('\s', '', compressed)  # remove any whitespace
    size = 0
    i = 0
    
    while i < len(compressed):
        match = matcher(compressed, i)
        if match:
            i = match.end(0)
            length, multiplier = map(int, match.groups())
            chunk = compressed[i: i + length]
            chunk_length = decompressed_length(chunk)

            size += chunk_length * multiplier
            i += length
        else:
            size += 1
            i += 1
    return size
    

assert decompressed_length('(3x3)ABCY') == len('ABCABCABCY')
assert decompressed_length('X(8x2)(3x3)ABCY') == len('XABCABCABCABCABCABCY')
assert decompressed_length('(27x12)(20x12)(13x14)(7x10)(1x12)A') == 241920

decompressed_length(Input(9).read())

11558231665

# [Day 10: Balance Bots](http://adventofcode.com/2016/day/10)


In [41]:
def follow_instructions(instructions, target):
    # the current position of each chip
    state = defaultdict(set)
    
    # bot chip movements
    give_instructions = {
        bot: (dest1, dest2)
        for (bot, dest1, dest2)
        in re.findall('(bot \d+) gives low to (\w+ \d+) and high to (\w+ \d+)', instructions)
    }
    
    def perform_give(bot, chip, receiver):
        """Bot gives chip to the receiver."""
        state[bot].discard(chip)  # remove if present, no keyerror
        state[receiver].add(chip)
        
        chips = state[receiver]
        
        # part 1 question
        if chips == target:
            print('{} is comparing {}'.format(receiver, target))
        
        if len(chips) == 2 and receiver in give_instructions:
            # Perform any give instructions required
            instruction = give_instructions[receiver]
            perform_give(receiver, min(chips), instruction[0])
            perform_give(receiver, max(chips), instruction[1])
    
    # Now we cycle through the input bins, giving them to bots
    for (chip, receiver) in re.findall('value (\d+) goes to (bot \d+)', instructions):
        chip = int(chip)
        perform_give('input', chip, receiver)
    
    return state
    
test_instructions = """
value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2
"""
test_state = follow_instructions(test_instructions, {2, 5})
assert test_state['output 0'] == {5}
assert test_state['output 1'] == {2}
assert test_state['output 2'] == {3}

state = follow_instructions(Input(10).read(), {17, 61})


bot 2 is comparing {2, 5}
bot 118 is comparing {17, 61}


The second portion simply requires us to multiple the values of outputs 1, 2 and 3

In [124]:
get_out = lambda index: int(state['output {}'.format(index)].pop())
get_out(0) * get_out(1) * get_out(2)

143153

# [Day 11: Radioisotope Thermoelectric Generators](http://adventofcode.com/2016/day/11)

In [98]:
floors = {0, 1, 2, 3}
State = namedtuple('State', 'elevator,floors')

        
def print_state(state):
    print_floors = []
    for floor_num in floors[::-1]:
        items = sorted(state.floors[floor_num])
        print_floors.append('F{} {} {}'.format(
            floor_num,
            'E' if floor_num == state.elevator else ' ',
            ' '.join(items)
        ))
    print('\n'.join(print_floors))


def available_combinations(state):
    """We have to move 1 item, and at most two, provide all possible variants."""
    floor = state.elevator
    items = state.floors[floor]
    combos = chain(
        combinations(items, 2),
        combinations(items, 1),
    )

    for combo in combos:
        yield frozenset(combo)


def available_floors(state):
    """Build the list of available elevator moves."""
    position = state.elevator
    for delta in (-1, 1):
        new_position = position + delta
        if new_position in floors:
            yield new_position


def valid_floor(floor):
    """Valid if zero genrators, or each chip has a generator."""
    generators = {item for item in floor if item[1] == 'G'}
    chips = {item for item in floor if item[1] == 'M'}
    
    return not generators or all(
        '{}G'.format(chip[0]) in floor
        for chip in chips
    )


def available_moves(state):
    """Find all the available moves for a state."""
    current_floor = state.elevator
    potential_floors = {
        current_floor + 1,
        current_floor - 1,
    }
    for combination in available_combinations(state):
        for new_floor in potential_floors & floors:
            # Build the new state
            new_floors = tuple(
                floor | combination if floor_num == new_floor else
                floor - combination if floor_num == current_floor else
                floor
                for floor_num, floor in enumerate(state.floors)
            )
            next_state = State(new_floor, new_floors)
            
            floors_to_check = (
                next_state.floors[new_floor],
                next_state.floors[current_floor]
            )
            if all(valid_floor(floor) for floor in floors_to_check):
                yield next_state
    

def heuristic(state):
    """Define when we're complete, 0 marking completion.
    
    Attempt to estimate the number of remainin moves.
    """
    top_floor = max(floors)
   
    return sum(
        len(state.floors[floor]) * (top_floor - floor)
        for floor in floors
    )


assert valid_floor([])
assert valid_floor(['HG'])
assert valid_floor(['HG', 'HM', 'LG', 'LM'])
assert valid_floor(['HM'])
assert not valid_floor(['HM', 'LG'])
assert not valid_floor(['HM', 'LG', 'LM'])



In [104]:
easy = State(
    elevator=0,
    floors=(
        frozenset(('RG', 'RM')),
        frozenset(),
        frozenset(),
        frozenset()
    )
)
% time a_star(easy, heuristic, available_moves)

CPU times: user 220 µs, sys: 1 µs, total: 221 µs
Wall time: 224 µs


[State(elevator=0, floors=(frozenset({'RG', 'RM'}), frozenset(), frozenset(), frozenset())),
 State(elevator=1, floors=(frozenset(), frozenset({'RG', 'RM'}), frozenset(), frozenset())),
 State(elevator=2, floors=(frozenset(), frozenset(), frozenset({'RG', 'RM'}), frozenset())),
 State(elevator=3, floors=(frozenset(), frozenset(), frozenset(), frozenset({'RG', 'RM'})))]

In [106]:
test_state = State(
    elevator=0,
    floors=(
        frozenset(('HM', 'LM')),
        frozenset(('HG', )),
        frozenset(('LG', )),
        frozenset(),
    )
)

% time path = a_star(test_state, heuristic, available_moves)
print(len(path) - 1)

CPU times: user 2.85 ms, sys: 238 µs, total: 3.09 ms
Wall time: 2.96 ms
11


I entered the test manually on this one, it's given as follows:
```
The first floor contains a polonium generator, a thulium generator, a thulium-compatible microchip, a promethium generator, a ruthenium generator, a ruthenium-compatible microchip, a cobalt generator, and a cobalt-compatible microchip.
The second floor contains a polonium-compatible microchip and a promethium-compatible microchip.
The third floor contains nothing relevant.
The fourth floor contains nothing relevant.
```

To avoid confusion with names, I'm using the following key:

```
    P = polonium
    T = thulium
    p = promethium
    R = ruthenium
```

In [107]:
state = State(
    elevator=0,
    floors=(
        frozenset(('PG', 'TG', 'TM', 'pG', 'RG', 'RM', 'CG', 'CM')),
        frozenset(('PM', 'pM')),
        frozenset(),
        frozenset(),                  
    )
)

% time path = a_star(state, heuristic, available_moves)
print(len(path) - 1)

CPU times: user 14.3 s, sys: 83.7 ms, total: 14.3 s
Wall time: 14.4 s
47


The second part has us add two additional sets of chips and generators to the ground floor.

In [353]:
state = State(
    elevator=0,
    floors=(
        frozenset(('PG', 'TG', 'TM', 'pG', 'RG', 'RM', 'CG', 'CM', 'EG', 'EM', 'DG', 'DM')),
        frozenset(('PM', 'pM')),
        frozenset(),
        frozenset(),                  
    )
)
% time path = a_star(state, heuristic, available_moves)
print(len(path) - 1)

Wall time: 21min 25s
71


Ouch, that took far too long. (Not to mention it ate over 4Gb of memory.)

The issue here is the number of braches we explore that are equivilent.

The easiest solution to this perhaps would be to use a custom state object that used a custom hash and equality operation to compare two objects?

i.e.

```
F4   LG
F3
F2   LM
F1 E PG PM
```

is the same as,

```
F4   PG
F3
F2   PM
F1 E LG LM
```

# [Day 12: Leonardo's Monorail](http://adventofcode.com/2016/day/12)

We've just beeing introduced to `assembunny`, a small set of 4 instructions that operate of 4 registers.

```
    cpy x y  Copies x into registry y (where x can be an int or a registry)
    inc x    Adds 1 to the value in registry x
    dec x    Adds -1 to the value in registry x
    jnz x y  Jumps y instructions away if x is non zero (where x can be an int or a registry)
```

In [227]:
def perform_instructions(instructions, regs):
    val = lambda a: regs.get(a, a)
    pos = 0
    
    while pos < len(instructions):
        instruction = instructions[pos]       
        op, x, y = instruction[0], instruction[1], instruction[-1]

        if op == 'cpy': regs[y] = val(x)
        elif op == 'inc': regs[x] += 1
        elif op == 'dec': regs[x] -= 1
        elif op == 'jnz' and val(x) != 0:
            pos += int(y)
            # as we're jumping, don't increment the position,
            # just skip the rest of the loop
            continue

        pos += 1
    

def parse_instructions(data):
    return tuple(
        tuple(
            part if part.isalpha() else int(part)
            for part in instruction.split(' ')
        )
        for instruction in data.split('\n')
        if instruction
    )

In [228]:
test_input = """
cpy 41 a
inc a
inc a
dec a
jnz a 2
dec a
"""
test_instructions = parse_instructions(test_input)

In [229]:
regs = dict(a=0, b=0, c=0, d=0)
perform_instructions(test_instructions, regs)
print(regs)

{'b': 0, 'd': 0, 'c': 0, 'a': 42}


In [231]:
regs = dict(a=0, b=0, c=0, d=0)
instructions = parse_instructions(Input(12).read())
perform_instructions(instructions, regs)
print(regs)

{'b': 196418, 'd': 0, 'c': 0, 'a': 318003}


For the second part I needed to initialize the `c` register with 1.

In [232]:
regs = dict(a=0, b=0, c=1, d=0)
perform_instructions(instructions, regs)
print(regs)

{'b': 5702887, 'd': 0, 'c': 0, 'a': 9227657}


# [Day 13: A Maze of Twisty Little Cubicles](http://adventofcode.com/2016/day/13)

In [256]:
def is_wall(seed, point):
    x, y = point
    val = (x*x + 3*x + 2*x*y + y + y*y) + seed
    num_ones = bin(val).count('1')
    return num_ones % 2 != 0


def available_moves(seed):
    def moves(point):
        x, y = point
        for point in neighbours4(x, y):
            new_x, new_y = point
            if new_x < 0 or new_y < 0:
                continue
            if is_wall(seed, point):
                continue
            yield(point)
    return moves

11


In [ ]:
path = a_star(
    (1, 1),
    lambda point: manhattan_distance(point, (7, 4)),
    available_moves(10),
)
print(len(path) - 1)

In [258]:
seed = 1352
target = (31, 39)
path = a_star(
    (1, 1),
    lambda point: manhattan_distance(point, target),
    available_moves(seed),
)
print(len(path) - 1)

90
